<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/ml_intro/blob/main/2_planificacion/3_dl/geron/10_chapter/pagina_506_3_ed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/ml_intro/blob/main/2_planificacion/3_dl/geron/10_chapter/pagina_506_3_ed.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

Esta es una traducción al español de las páginas **507 y 508** del libro *"Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow"* de Aurélien Géron, sección **"Building a Regression MLP Using the Sequential API"**:





# **Construyendo un MLP de Regresión Usando la API Secuencial**  


Volvamos al problema de la vivienda en California (*California housing*) y abordémoslo usando el mismo **MLP** (*Multi-Layer Perceptron*) que antes, con **3 capas ocultas de 50 neuronas cada una**, pero esta vez construyéndolo con Keras.  



Usar la **API Secuencial** para construir, entrenar, evaluar y utilizar un MLP de regresión es muy similar a lo que hicimos para clasificación. Las principales diferencias en el siguiente ejemplo son:  


1. **Capa de salida**: Tiene una sola neurona (ya que solo predecimos un valor) y **no usa función de activación**.  


2. **Función de pérdida**: Error cuadrático medio (*mean squared error*, MSE). 

 
3. **Métrica**: RMSE (*Root Mean Squared Error*).  


4. **Optimizador**: Usamos **Adam**, igual que `MLPRegressor` de Scikit-Learn.  



Además, en este ejemplo:  


- No necesitamos una capa `Flatten`.  
- Usamos una capa de **Normalización** como primera capa: hace lo mismo que `StandardScaler` de Scikit-Learn, pero debe ajustarse a los datos de entrenamiento con su método `adapt()` antes de llamar a `fit()`. (Keras tiene otras capas de preprocesamiento, que se cubrirán en el Capítulo 13).  



#### **Código Ejemplo**:

Aquí tienes las líneas de código específicas para:  
 
* **obtener los datos, 
* dividirlos en predictores (X) y etiquetas (y)**, y 
* luego separarlos en conjuntos de **entrenamiento, validación y prueba**   
 
para el ejemplo del MLP secuencial con Keras que tradujimos anteriormente:



---

### **1. Obtener el Dataset (California Housing)**


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


In [ ]:

# Cargar el dataset completo
housing = fetch_california_housing()


In [ ]:
type(housing) # La estructura es un objeto de tipo Bunch, similar a un diccionario

In [ ]:

# Separar predictores (X) y etiquetas (y)
X, y = housing.data, housing.target  # X.shape = (20640, 8), y.shape = (20640,)





# **2. Dividir en Train (70%), Validación (15%) y Test (15%)**


In [ ]:
# Primera división: Train (70%) y Temporal (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42  # Semilla para reproducibilidad
)


In [ ]:

# Segunda división: Validación (15%) y Test (15%) del temporal
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.5,  # Divide el 30% en 15% validación y 15% test
    random_state=42
)




### **3. Verificación de las Dimensiones**


In [ ]:
print(f"Train: {X_train.shape}, {y_train.shape}")      # (14448, 8), (14448,)
print(f"Validación: {X_valid.shape}, {y_valid.shape}")  # (3096, 8), (3096,)
print(f"Test: {X_test.shape}, {y_test.shape}")          # (3096, 8), (3096,)






### **Explicación Clave**:


- **`random_state=42`**: Garantiza que la división sea reproducible (misma división en cada ejecución).  
- **Proporciones**:  
  - **70% entrenamiento**: Para aprender patrones.  
  - **15% validación**: Para ajustar hiperparámetros y evitar overfitting.  
  - **15% test**: Para evaluar el modelo final de manera imparcial.  





### **Nota sobre Preprocesamiento**:
El texto de Géron usa una **capa de Normalización de Keras** (no `StandardScaler`), por lo que no escalamos manualmente los datos. La capa se ajustará con `.adapt()` antes del entrenamiento:


In [ ]:
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
norm_layer.adapt(X_train)  # Ajuste a los datos de entrenamiento
# Normalizar los conjuntos de datos



# **¿Por qué esta división?**  
- **Train**: Mayor porcentaje para que el modelo aprenda bien.  
- **Validación**: Detecta overfitting durante el entrenamiento (ej.: con `validation_data` en `model.fit()`).  
- **Test**: Simula datos nunca vistos para evaluar el rendimiento real.  
- **Proporciones**:  
  - **70% entrenamiento**: Para aprender patrones.  
  - **15% validación**: Para ajustar hiperparámetros y evitar overfitting.  
  - **15% test**: Para evaluar el modelo final de manera imparcial.


Si ejecutas este código antes del modelo secuencial de Géron, tendrás los datos listos para entrenar el MLP.

In [ ]:
import tensorflow as tf

# Fijar semilla para reproducibilidad
tf.random.set_seed(42)


In [ ]:

# Capa de Normalización (equivalente a StandardScaler)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])


In [ ]:
X_train.shape[1:] 

In [ ]:

# Modelo Secuencial
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)  # Sin activación para regresión
])


In [ ]:

# Optimizador y compilación
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])


In [ ]:

%%time 
# Ajustar normalización y entrenar
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))


In [ ]:

# Evaluación y predicción
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)
y_pred, y_test[:3]  # Comparar predicciones con etiquetas reales



## **Nota Importante**:  
La capa de `Normalization` aprende las **medias y desviaciones estándar** de las características al llamar a `adapt()`. Sin embargo, en el resumen del modelo, estos parámetros aparecen como **no entrenables** (*non-trainable*), porque no son afectados por el descenso de gradiente.  





### **Ventajas y Limitaciones de la API Secuencial**  
Como ves, la **API Secuencial** es clara y sencilla. No obstante, aunque los modelos secuenciales son muy comunes, a veces es necesario construir redes con:  
- Topologías más complejas.  
- Múltiples entradas o salidas.  

Para estos casos, Keras ofrece la **API Funcional** (que se explicará más adelante).  

---



### **Traducción de Términos Clave**:  
| Inglés | Español |  
|--------|---------|  
| Hidden layers | Capas ocultas |  
| Output layer | Capa de salida |  
| Mean squared error (MSE) | Error cuadrático medio |  
| Root Mean Squared Error (RMSE) | Raíz del error cuadrático medio |  
| Adam optimizer | Optimizador Adam |  
| Flatten layer | Capa de aplanamiento |  
| Normalization layer | Capa de normalización |  
| Non-trainable parameters | Parámetros no entrenables |  



Esta traducción conserva el **tono técnico** del original y adapta los conceptos para hispanohablantes, manteniendo los términos clave en inglés entre paréntesis cuando es relevante. 

# Práctica de codificación 

Vamos a diseñar una **práctica de codificación paso a paso** basada en el texto de Géron, centrada en desarrollar un **MLP para regresión** con Keras (como en el ejemplo del libro), pero añadiendo desafíos para fortalecer tu **pensamiento computacional**. Aprenderás a:  



1. **Preprocesar datos** (normalización).  
2. **Construir y entrenar un modelo secuencial**.  
3. **Evaluar y ajustar hiperparámetros**.  
4. **Analizar resultados** (visualización de métricas).  





# **Práctica: MLP para Predecir Precios de Viviendas en California**  


**Dataset**: Usaremos el mismo dataset que Géron (*California Housing*), disponible en `sklearn.datasets`.



#### **Paso 1: Configuración del Entorno**  


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
# Se tarda alrededor de 9 segundos 

In [2]:
%%time 
# Cargar datos
housing = fetch_california_housing()


CPU times: total: 0 ns
Wall time: 13 ms


In [3]:
housing.keys() 

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [4]:
# dividir en predictores y etiquetas 
X, y = housing.data, housing.target


In [6]:
y 

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [7]:
%%time 
# Dividir en train (70%), validación (15%), test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


CPU times: total: 0 ns
Wall time: 8.01 ms




#### **Paso 2: Preprocesamiento con Normalización**  
Aquí compararemos dos enfoques:  
- **Normalización con Keras** (como en Géron).  
- **StandardScaler de Scikit-Learn** (para entender diferencias).  


In [8]:
# Opción 1: Normalización con Keras
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
norm_layer.adapt(X_train)  # Ajuste a datos de entrenamiento


c:\Users\marco\Documentos\investigacion\ml_intro\.venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:

# Opción 2: StandardScaler de Scikit-Learn
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)




#### **Paso 3: Construcción del Modelo Secuencial**  


In [10]:
def build_model(normalization_layer=None):
    model = tf.keras.Sequential()
    if normalization_layer:
        model.add(normalization_layer)  # Usar capa de Keras
    model.add(tf.keras.layers.Dense(50, activation="relu"))
    model.add(tf.keras.layers.Dense(50, activation="relu"))
    model.add(tf.keras.layers.Dense(50, activation="relu"))
    model.add(tf.keras.layers.Dense(1))  # Salida lineal para regresión
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(loss="mse", metrics=["RootMeanSquaredError"])
    return model


In [11]:

%%time 
# Modelo con normalización de Keras
model_keras = build_model(norm_layer)
history_keras = model_keras.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 1.0970 - loss: 1.2904 - val_RootMeanSquaredError: 0.6712 - val_loss: 0.4505
Epoch 2/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.6346 - loss: 0.4028 - val_RootMeanSquaredError: 0.6139 - val_loss: 0.3768
Epoch 3/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.6005 - loss: 0.3607 - val_RootMeanSquaredError: 0.6010 - val_loss: 0.3612
Epoch 4/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - RootMeanSquaredError: 0.5880 - loss: 0.3458 - val_RootMeanSquaredError: 0.6031 - val_loss: 0.3637
Epoch 5/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - RootMeanSquaredError: 0.5789 - loss: 0.3352 - val_RootMeanSquaredError: 0.5869 - val_loss: 0.3445
Epoch 6/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - RootMeanSquaredError: 0.5690 - loss: 0.3238 - val_RootMeanSquaredError: 0.5710 - val_loss: 0.3260
Epoch 7/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - RootMeanSquaredError: 0.5662 - los

In [12]:
%%time 
# Modelo con StandardScaler
model_scaler = build_model()
history_scaler = model_scaler.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_valid_scaled, y_valid))



Epoch 1/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 1.0611 - loss: 1.1991 - val_RootMeanSquaredError: 0.6837 - val_loss: 0.4674
Epoch 2/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6241 - loss: 0.3896 - val_RootMeanSquaredError: 0.6598 - val_loss: 0.4353
Epoch 3/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6041 - loss: 0.3650 - val_RootMeanSquaredError: 0.6136 - val_loss: 0.3765
Epoch 4/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5893 - loss: 0.3474 - val_RootMeanSquaredError: 0.5945 - val_loss: 0.3535
Epoch 5/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5802 - loss: 0.3368 - val_RootMeanSquaredError: 0.5814 - val_loss: 0.3380
Epoch 6/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5705 - loss: 0.3257 - val_RootMeanSquaredError: 0.5920 - val_loss: 0.3505
Epoch 7/20
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5604 - los


#### **Paso 4: Evaluación y Visualización** 

In [ ]:
# Función para graficar curvas de aprendizaje
def plot_loss(history, title):
    plt.plot(history.history["loss"], label="Train")
    plt.plot(history.history["val_loss"], label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("MSE")
    plt.title(title)
    plt.legend()
    plt.grid(alpha = 0.3)
    plt.xticks(np.arange(0, 21, 1))
    plt.yticks(np.arange(0.20, 0.90, 0.05))
    plt.savefig(rf"C:\Users\marco\Downloads\{title}.png")
    plt.show()


NameError: name 'history_keras' is not defined

In [3]:

plot_loss(history_keras, "Normalización con Keras")


NameError: name 'history_keras' is not defined

In [4]:
plot_loss(history_scaler, "Normalización con StandardScaler")


NameError: name 'history_scaler' is not defined

In [18]:

# Evaluar en test
mse_keras, rmse_keras = model_keras.evaluate(X_test, y_test)
mse_scaler, rmse_scaler = model_scaler.evaluate(X_test_scaled, y_test)
print(f"Keras Normalization - Test RMSE: {rmse_keras:.4f}")
print(f"StandardScaler - Test RMSE: {rmse_scaler:.4f}")



97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - RootMeanSquaredError: 0.5418 - loss: 0.2938
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - RootMeanSquaredError: 0.5217 - loss: 0.2724
Keras Normalization - Test RMSE: 0.5486
StandardScaler - Test RMSE: 0.5124



#### **Paso 5: Desafíos para Pensamiento Computacional**  
1. **Experimenta con Arquitecturas**:  
   - ¿Qué pasa si cambias el número de neuronas (ej.: 30 en lugar de 50)?  
   - Prueba añadir una cuarta capa oculta.  



2. **Ajuste de Hiperparámetros**:  
   - Modifica el `learning_rate` del optimizador Adam (prueba 1e-2, 1e-4).  
   - Usa **early stopping** para evitar sobreajuste: 

In [19]:
 

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(..., callbacks=[early_stopping])



NameError: name 'model' is not defined


3. **Compara Preprocesadores**:  
   - ¿Cuál método de normalización da mejores resultados? ¿Por qué?  



4. **Predicciones Cualitativas**:  
   - Imprime las primeras 5 predicciones del modelo y compáralas con los valores reales:

In [20]:
  
y_pred = model_keras.predict(X_test[:5])
print("Predicciones:", y_pred.flatten())
print("Valores Reales:", y_test[:5])



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Predicciones: [0.85819227 1.1058774  3.7119164  1.2053517  1.7731631 ]
Valores Reales: [1.    1.188 3.761 2.    0.952]



### **Resultado Esperado**  


- Aprenderás a **debuggear modelos**: Si el RMSE es muy alto, revisa si los datos están bien normalizados.  
- Entenderás el impacto de **la arquitectura y el learning rate** en el entrenamiento.  
- Visualizarás el **sobreajuste** (si el loss de validación sube en epochs altas).  



### **Conclusión**  
Esta práctica refleja el **flujo de trabajo real en ML**:  


1. Preprocesar datos → 2. Construir modelo → 3. Entrenar/Ajustar → 4. Evaluar.  
**Tips adicionales**:  
- Usa `model.summary()` para ver la estructura del modelo.  
- Explora `tf.keras.utils.plot_model()` para visualizar la arquitectura.  


In [1]:
tf.keras.utils.plot_model(model_keras, show_shapes=True, to_file='model_keras.png')

NameError: name 'tf' is not defined